In [1]:
import cv2
#from PIL import Image
import numpy as np
import dlib
#import os
from imutils import face_utils
from pygame import mixer
import time
mixer.init()
sound = mixer.Sound('beep-01a.wav')
sound1 = mixer.Sound('audio1.wav')
import requests

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
TELEGRAM_BOT_TOKEN = 'telegram bot token'
TELEGRAM_CHAT_ID = 'telegram chat id '



def send_telegram_message(message):
    send_text = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage?chat_id={TELEGRAM_CHAT_ID}&parse_mode=Markdown&text={message}'
    response = requests.get(send_text)
    return response.json()




In [11]:
capture = cv2.VideoCapture(1)


In [12]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [13]:
#Id=input('enter your id')
#sampleNum=0

In [14]:
sleep = 0
drowsy = 0
active = 0
status=""
color=(255,0,255)

In [15]:
def compute(ptA,ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

In [16]:
def blinked(a,b,c,d,e,f):
    up = compute(b,d) + compute(c,e)
    down = compute(a,f)
    ratio = up/(2.0*down)

    
    if(ratio>0.25):
        return 2
    elif(ratio>0.21 and ratio<=0.25):
        return 1
    else:
        return 0

In [17]:
while True:
    ret, frame = capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 5)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)


        left_blink = blinked(landmarks[36],landmarks[37], 
            landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42],landmarks[43], 
            landmarks[44], landmarks[47], landmarks[46], landmarks[45])



        if(left_blink==0 or right_blink==0):
            sleep+=1
            drowsy=0
            active=0
            if(sleep>6):
                status="SLEEPING zZZ!!"
                color = (255,125,14)
                send_telegram_message("Warning! , the student is sleeping.")
                
                try:
                    sound.play()
                except: 
                    isplaying = False
                    pass
                    
        elif(left_blink==1 or right_blink==1):
            sleep=0
            active=0
            drowsy+=1
            if(drowsy>6):
                status="Drowsy !!!"
                color = (16,79,255)
                send_telegram_message("Please wake up!!!!!!!!")
                try:
                    sound1.play()
                except: 
                    isplaying = False
                    pass

        else:
            drowsy=0
            sleep=0
            active+=1
            if(active>6):
                status="Active :)"
                color = (133,255,28)
            #print(status);

        cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, color,5)

        for n in range(0, 68):
            (x,y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)
    #cv2.imwrite("dataSet/User."+Id +'.'+ str(sampleNum) + ".jpg", gray[y:y2,x:x2])
    #sampleNum=sampleNum+
    cv2.imshow("Drowsiness detection", frame)
    #cv2.imshow("Result of detector", face_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()
capture.release()